In [1]:
:ext QuasiQuotes

In [2]:
import qualified H.Prelude as H
H.initialize H.defaultConfig

In [3]:
[rprint|
  library(keras)
  library(dplyr)
  library(ggplot2) |] 

 [1] "ggplot2"   "dplyr"     "keras"     "stats"     "graphics"  "grDevices"
 [7] "utils"     "datasets"  "methods"   "base"

In [4]:
[rprint| reticulate::py_config() |]  

python:         /nix/store/5kfn0xxh3ipjdyjly2d5wrmh4cidsm8k-python3-3.5.3/bin/python
libpython:      /nix/store/5kfn0xxh3ipjdyjly2d5wrmh4cidsm8k-python3-3.5.3/lib/libpython3.5m.so
pythonhome:     /nix/store/5kfn0xxh3ipjdyjly2d5wrmh4cidsm8k-python3-3.5.3:/nix/store/5kfn0xxh3ipjdyjly2d5wrmh4cidsm8k-python3-3.5.3
version:        3.5.3 (default, Jan 17 2017, 07:57:56)  [GCC 5.4.0]
numpy:          /nix/store/7xk7ylqagd3w72y1p39rlcsamx3hcxl3-python3.5-numpy-1.12.1/lib/python3.5/site-packages/numpy
numpy_version:  1.12.1
tensorflow:     /nix/store/qmr3h3f1s1x1dr64sb61igig8469imn6-python3.5-tensorflow-1.1.0/lib/python3.5/site-packages/tensorflow

python versions found: 
 /nix/store/5kfn0xxh3ipjdyjly2d5wrmh4cidsm8k-python3-3.5.3/bin/python
 /usr/bin/python
 /usr/bin/python3

In [6]:
[rprint| 
 
  # model
  set.seed(22222)
  model <- keras_model_sequential() 
  model %>% 
    layer_lstm(units = 4, input_shape = c(5, 1)) %>% 
    layer_dense(units = 1) %>% 
    compile(
      loss = 'mean_squared_error',
      optimizer = 'adam'
    )
  
  model %>% summary()  |]

 

Model
________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
lstm_1 (LSTM)                       (None, 4)                       96          
________________________________________________________________________________
dense_1 (Dense)                     (None, 1)                       5           
Total params: 101.0
Trainable params: 101
Non-trainable params: 0.0
________________________________________________________________________________

 
NULL

In [7]:
[rprint| 
  lstm_num_timesteps <- 4 # one less now
  batch_size <- 1
  epochs <- 5
  lstm_units <- 4

  #
  trend_train <- 11:110 + rnorm(100, sd = 2)
  trend_test <- 111:130 + rnorm(20, sd =2)
  # difference
  trend_train_start <- trend_train[1]
  trend_test_start <- trend_test[1]
  trend_train_diff <- diff(trend_train)
  trend_test_diff <- diff(trend_test)
  # normalize
  minval <- min(trend_train_diff)
  maxval <- max(trend_train_diff)
  normalize <- function(vec, min, max) {
  (vec-min) / (max-min)
  }
  denormalize <- function(vec,min,max) {
  vec * (max - min) + min
  }
  trend_train_diff <- normalize(trend_train_diff, minval, maxval)
  trend_test_diff <- normalize(trend_test_diff, minval, maxval)


  # get data into "timesteps form"
  X_train <- t(sapply(1:(length(trend_train_diff) - lstm_num_timesteps), function(x) trend_train_diff[x:(x + lstm_num_timesteps - 1)]))
  y_train <- sapply((lstm_num_timesteps + 1):(length(trend_train_diff)), function(x) trend_train_diff[x])
  X_test <- t(sapply(1:(length(trend_test_diff) - lstm_num_timesteps), function(x) trend_test_diff[x:(x + lstm_num_timesteps - 1)]))
  y_test <- sapply((lstm_num_timesteps + 1):(length(trend_test_diff)), function(x) trend_test_diff[x])

  # Keras LSTMs expect the input array to be shaped as (no. samples, no. time steps, no. features)
  dim(X_train) <- c(dim(X_train)[1], dim(X_train)[2], 1)
  dim(X_train)
  num_samples <- dim(X_train)[1]
  num_steps <- dim(X_train)[2]
  num_features <- dim(X_train)[3]
  c(num_samples, num_steps, num_features)

  dim(X_test) <- c(dim(X_test)[1], dim(X_test)[2], 1)

  # model
  set.seed(22222)
  model <- keras_model_sequential() 
  model %>% 
    layer_lstm(units = lstm_units, input_shape = c(num_steps, num_features)) %>% 
    layer_dense(units = 1) %>% 
    compile(
      loss = 'mean_squared_error',
      optimizer = 'adam'
    )
  
  model %>% summary()
  
  model %>% fit( 
    X_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = list(X_test, y_test)
  )
  
  # model %>% save_model_hdf5(filepath = paste0(model_name, ".h5"))
 
  pred_train <- model %>% predict(X_train, batch_size = 1)
  pred_test <- model %>% predict(X_test, batch_size = 1)

  pred_train <- denormalize(pred_train, minval, maxval)
  pred_test <- denormalize(pred_test, minval, maxval)

  pred_train_undiff <- pred_train + trend_train[(lstm_num_timesteps+1):(length(trend_train)-1)]
  pred_test_undiff <- pred_test + trend_test[(lstm_num_timesteps+1):(length(trend_test)-1)]  |]

 

Model
________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
lstm_2 (LSTM)                       (None, 4)                       96          
________________________________________________________________________________
dense_2 (Dense)                     (None, 1)                       5           
Total params: 101.0
Trainable params: 101
Non-trainable params: 0.0
________________________________________________________________________________

 
Train on 95 samples, validate on 15 samples
Epoch 1/5
95/95 [==============================] - 1s - loss: 0.1806 - val_loss: 0.0790
Epoch 2/5
95/95 [==============================] - 0s - loss: 0.0503 - val_loss: 0.0526
Epoch 3/5
95/95 [==============================] - 0s - loss: 0.0452 - val_loss: 0.0518
Epoch 4/5
95/95 [==============================] - 0s - loss: 0.0444 - val_loss: 0.0516
Epoch 5/5
95/95 [========================

In [ ]:
 #df <- data_frame(time_id = 1:120,
  #               train = c(trend_train, rep(NA, length(trend_test))),
  #               test = c(rep(NA, length(trend_train)), trend_test),
  #               pred_train = c(rep(NA, lstm_num_timesteps+1), pred_train_undiff, rep(NA, length(trend_test))),
  #               pred_test = c(rep(NA, length(trend_train)), rep(NA, lstm_num_timesteps+1), pred_test_undiff))
  #df <- df %>% gather(key = 'type', value = 'value', train:pred_test)
  #ggplot(df, aes(x = time_id, y = value)) + geom_line(aes(color = type))

In [ ]:
[rprint| 1 + 1 |]

In [ ]:
[r| fact <<- function(n) if(n == 0) 1 else n * fact(n - 1) |]

In [ ]:
[rprint| fact(10) |]

In [ ]:
[rgraph| plot(cars) |]

In [ ]:
[rgraph| plot(cars); abline(lm(cars$dist ~ cars$speed), col="red") |]

In [ ]:
[r| require("ggplot2") |]

In [ ]:
[rgraph|
  Xv <- c(rnorm (500, 10,3), rnorm (500, 50, 20), rnorm (500, 70, 20))
  Yv <- c(rnorm (500, 10,3), rnorm (500, 70, 5), rnorm (500, 30, 5))
  myd <- data.frame(Xv, Yv)

  ggplot(myd, aes(x = Xv, y = Yv)) + geom_point() + geom_density2d() + theme_bw() |]